In [1]:
%sql show tables

In [2]:
%sql drop table tmp

In [3]:
%sql create table tmp as select c1||':'||c2||':'||c3||':'||c4||':'||c5||':'||c6||':'||c7 as col FROM weblog

In [4]:
%sql select * from tmp

In [5]:
# create pyspark dataframe from tmp table
df = spark.sql("SELECT col FROM tmp")

In [6]:
# display pyspark dataframe
display(df)

In [7]:
# use regex to separate the fields
from pyspark.sql.functions import *
df = df.select(regexp_replace(col('col'), ' "', ',"'))
df = df.select(col("regexp_replace(col,  \", ,\")").alias("col"))
df = df.select(regexp_replace(col('col'), '" ', '",'))
df = df.select(col("regexp_replace(col, \" , \",)").alias("col"))

In [8]:
# split the flields 1
split_col = split(df['col'], ',"')
df = df.withColumn('c1', split_col.getItem(0))
df = df.withColumn('request', split_col.getItem(1))
df = df.withColumn('c3', split_col.getItem(2))

In [9]:
# split the flields 2
split_col = split(df['c3'], '",')
df = df.withColumn('user_agent', split_col.getItem(0))
df = df.withColumn('c5', split_col.getItem(1))

In [10]:
# split the flields 3
split_col = split(df['c5'], ' ')
df = df.withColumn('ssl_cipher', split_col.getItem(0))
df = df.withColumn('ssl_protocol', split_col.getItem(1))

In [11]:
# split the flields 4
split_col = split(df['c1'], ' ')
df = df.withColumn('timestamp', split_col.getItem(0))
df = df.withColumn('elb', split_col.getItem(1))
df = df.withColumn('client_port', split_col.getItem(2))
df = df.withColumn('backend_port', split_col.getItem(3))
df = df.withColumn('request_processing_time', split_col.getItem(4))
df = df.withColumn('backend_processing_time', split_col.getItem(5))
df = df.withColumn('response_processing_time', split_col.getItem(6))
df = df.withColumn('elb_status_code', split_col.getItem(7))
df = df.withColumn('backend_status_code', split_col.getItem(8))
df = df.withColumn('received_bytes', split_col.getItem(9))

In [12]:
# drop the unprocessed fields
from functools import reduce
from pyspark.sql import DataFrame

df = reduce(DataFrame.drop, ['c1','c3','c5'], df)

In [13]:
# display the dataframe
display(df)